# Notebook to explore updating PNAS data
- rewrite code to make json not csv <br>
- what are we dropping city year combos in pdays routine?

In [63]:
#### Dependencies
import pandas as pd
import numpy as np
import xarray as xr
from random import random
from itertools import groupby
from operator import itemgetter
import geopandas as gpd 
from glob import glob
from statistics import mean
import julian
import time 
import multiprocessing as mp 
from multiprocessing import Pool
import os

In [2]:
# Data path
DATA_PATH = '/home/cascade/projects/UrbanHeat/data/'


### data check

In [3]:
# Explore files - 1983
content = os.listdir(DATA_PATH+'interim/')
fns = sorted(glob(DATA_PATH+'interim/ERA5_HI/*csv'))
fn = fns[0]
df1983 = pd.read_csv(fn)
df1983.head()

,Unnamed: 0,ID_HDC_G0,CTR_MN_NM,1983.01.01,1983.01.02,1983.01.03,1983.01.04,1983.01.05,1983.01.06,1983.01.07,...,1983.12.22,1983.12.23,1983.12.24,1983.12.25,1983.12.26,1983.12.27,1983.12.28,1983.12.29,1983.12.30,1983.12.31
0,0,5782,Russia,-49.590861,-38.042995,-36.750948,-28.374049,-26.626346,-21.826344,-21.446321,...,-27.149449,-36.333571,-30.164864,-19.821702,-14.509835,-14.259680,-16.596240,-14.705007,-14.277887,-16.171796
1,1,3316,Russia,-4.088565,-2.252119,-6.184317,-13.705580,-7.631833,-7.392443,-5.487767,...,-12.845948,-18.931796,-8.486376,-8.990228,-15.775358,-19.373392,-19.813643,-12.648720,-13.315901,-15.861366
2,2,5645,Russia,-38.693309,-23.243517,-13.361891,-11.548153,-16.044299,-18.756276,-24.426800,...,-20.344301,-26.453421,-16.921110,-2.669969,-4.402858,-9.637487,-11.979378,-15.932049,-15.186321,-12.247702
3,3,3185,Finland,-0.263081,0.372479,-5.046791,-2.224906,-1.583173,-0.187118,-0.431349,...,-6.141768,-3.382719,-2.959680,-7.515815,-13.457510,-4.966569,-5.707407,-10.071544,-9.986938,-10.397102
4,4,3539,Russia,-10.146690,-9.364139,-20.047189,-16.052717,-12.344176,-5.424159,-0.371261,...,-5.950157,-2.916705,-1.734350,-1.815293,-6.173747,-10.115407,-10.211072,-10.359057,-10.359030,-11.863613


In [4]:
df1983.head()

,Unnamed: 0,ID_HDC_G0,CTR_MN_NM,1983.01.01,1983.01.02,1983.01.03,1983.01.04,1983.01.05,1983.01.06,1983.01.07,...,1983.12.22,1983.12.23,1983.12.24,1983.12.25,1983.12.26,1983.12.27,1983.12.28,1983.12.29,1983.12.30,1983.12.31
0,0,5782,Russia,-49.590861,-38.042995,-36.750948,-28.374049,-26.626346,-21.826344,-21.446321,...,-27.149449,-36.333571,-30.164864,-19.821702,-14.509835,-14.259680,-16.596240,-14.705007,-14.277887,-16.171796
1,1,3316,Russia,-4.088565,-2.252119,-6.184317,-13.705580,-7.631833,-7.392443,-5.487767,...,-12.845948,-18.931796,-8.486376,-8.990228,-15.775358,-19.373392,-19.813643,-12.648720,-13.315901,-15.861366
2,2,5645,Russia,-38.693309,-23.243517,-13.361891,-11.548153,-16.044299,-18.756276,-24.426800,...,-20.344301,-26.453421,-16.921110,-2.669969,-4.402858,-9.637487,-11.979378,-15.932049,-15.186321,-12.247702
3,3,3185,Finland,-0.263081,0.372479,-5.046791,-2.224906,-1.583173,-0.187118,-0.431349,...,-6.141768,-3.382719,-2.959680,-7.515815,-13.457510,-4.966569,-5.707407,-10.071544,-9.986938,-10.397102
4,4,3539,Russia,-10.146690,-9.364139,-20.047189,-16.052717,-12.344176,-5.424159,-0.371261,...,-5.950157,-2.916705,-1.734350,-1.815293,-6.173747,-10.115407,-10.211072,-10.359057,-10.359030,-11.863613


### Let's check out the functions

In [5]:
#### Step 1 - Function Loads all Tmax Data as an X-array
def read_data(dir_path, space_dim, time_dim):
    """ Function reads in all Tmax .csv files, joins them by date along the x-axis
    and returns the whole record as a x-array data array
    
    Args:   
        dir_path = path to .csv files 
        time_dim = name for time dim as a str ... use date :-)
        space_dim = col name for GHS-UCDB IDs as an str (ID_HDC_G0)
    """
    fn_list = sorted(glob.glob(dir_path+'*.csv'))
    df_out = pd.DataFrame()
    date_list = []

    # Open all Tmax files and concat into a df
    for i, fn in enumerate(fn_list):    
        # Open the CSV
        df = pd.read_csv(fn)

        # Get the city ids 
        if i == 1:
            df_id = df[space_dim]

        # get only the Tmax columns and concate date list 
        df_temp = df.iloc[:,3:] # get only temp columns
        date_list = date_list+list(df_temp.columns)

        # Drop cities w/ no temp record 
        df_temp_drop = df_temp.dropna()

        # Merge
        df_out = pd.concat([df_out, df_temp_drop], axis=1)
        print(df_out.shape)
    
    # make date into an array
    tmax_arr = df_out.to_numpy()

    # Make data into an xr.DataArray
    tmax_xr_da = xr.DataArray(tmax_arr, coords=[df_id, date_list], 
                             dims=[space_dim, time_dim])
    return tmax_xr_da

In [6]:
def tmax_days(xarray, Tthresh):
    """ Function finds all the tmax days in a year and sums total days per year 
    greater than a threshold within a year where Tmax > Tthresh for each city. Returns the total number of days,
    the dates, the tempatures, and the intensity (daily Tmax - Tthresh)
    
    Args: 
        xarray = an xarray object with dims = (space, times)
        Tthresh = int of temp threshold
    """
    
    # empty lists & df
    id_list = []
    date_list = []
    tmax_list = []
    intensity_list = []
    df_out = pd.DataFrame()
    
    # subset xarray
    out = xarray.where(xarray > Tthresh, drop = True)

    # start loop 
    for index, loc in enumerate(out.ID_HDC_G0):
        id_list.append(out.ID_HDC_G0.values[index]) # get IDS
        date_list.append(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').date.values) # get event dates
        
        # #CPT 2020.02.23 
        # dayTot_list.append(len(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').date.values)) # get event totals
        
        tmax_list.append(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').values) # get temp values
        intensity_list.append(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').values - Tthresh) # get severity

    # write to a data frame
    df_out['ID_HDC_G0'] = id_list
    # df_out['total_days'] = dayTot_list #CPT 2020.02.23
    df_out['dates'] = date_list
    df_out['tmax'] = tmax_list
    df_out['tmax_tntensity'] = intensity_list

    # return df_out
    return df_out

In [10]:
def event_split(dates, ID_HDC_G0, intensity, tmax): #, total_days): #CPT 2020.02.23
    
    """ Searchs a list of dates and isolates sequential dates as a list, then calculates event stats.
    See comments in code for more details. 
    
    Args:
        dates: pandas.core.index as julian dates
        ID_HDC_G0: city ID as string
        intensity: numpy.ndarray of intensities values
        tmax: numpy.ndarray of intensities values of tmax values
        total_days: total number of tmax days in a year for a given city

    """

    # city id
    city_id = ID_HDC_G0
    # tot_days = total_days #CPT 2020.02.23
    
    # lists to fill
    city_id_list = []
    # tot_days_list = [] #CPT 2020.02.23
    event_dates_list = []
    dur_list = []
    intensity_list = []
    tmax_list = []
    avg_temp_list = []
    avg_int_list = []
    tot_int_list = []
    year_list = []
    
    # data frame out
    df_out = pd.DataFrame()
    
    # turn days into julian days
    jul_days = jul_convert(dates)
    
    # Counters to make sure we write the correct event dates to a list, don't want julian days in output
    counter = 0
    start = 0
    end = 0
    
    # Loop through dur list and isolate seq days, temps, and intensities
    for k, g in groupby(enumerate(jul_days.values), lambda x: x[1]-x[0]):
        
        seq = list(map(itemgetter(1), g)) # isolate seq. days
        dur = len(seq) # duration of each event
        
        counter = counter + dur # add duration to counter
        end = counter # end of current event
        
        event_dates = dates[start:end] # dates of tmax days during each event
        intense = intensity[start:end] # intensity of each day during event
        temp = tmax[start:end] # temp of each day during event
        avg_temp = mean(temp) # avg. temp during event
        avg_int = mean(intense) # avg. intensity during event
        tot_int = intense.sum() # total intensity during event
        
        start = counter # reset start to current end (e.g. counter)
        year = event_dates[0].split('.')[0]
        
        # fill lists
        city_id_list.append(city_id)
        year_list.append(year)
        # tot_days_list.append(tot_days) #CPT 2020.02.23
        dur_list.append(dur)
        event_dates_list.append(event_dates)
        intensity_list.append(intense)
        tmax_list.append(temp)
        avg_temp_list.append(avg_temp)
        avg_int_list.append(avg_int)
        tot_int_list.append(tot_int)

    # write out as a dateframe
    df_out['ID_HDC_G0'] = city_id_list
    df_out['year'] = year_list
    # df_out['total_days'] = tot_days_list #CPT 2020.02.23
    df_out['duration'] = dur_list
    df_out['avg_temp'] = avg_temp_list
    df_out['avg_intensity'] = avg_int_list
    df_out['tot_intensity'] = tot_int_list
    df_out['event_dates'] = event_dates_list
    df_out['duration'] = dur_list
    df_out['intensity'] = intensity_list
    df_out['tmax'] = tmax_list

    return df_out

In [12]:
#### Step 4 function feeds output from function 3 into function 4
def tmax_stats(df_in):
    """ runs event_split functionon a dataframe to produce desired tmax stats

        NOTE - If you add arguments to event_split to make more states,
        be sure to update this function

        args:
            df: input dataframe
    """
    df_out = pd.DataFrame()

    # NOTE - If you add arguments to event_split to make more stats,
    # be sure to update this function

    for index, row in df_in.iterrows():
        dates = row['dates'] # Get event dates
        intensity = row['tmax_tntensity'] # Get intensity for each day
        tmax = row['tmax'] # Get tmax for each day
        ID_HDC_G0 = row['ID_HDC_G0'] # get city id
        # total_days = row['total_days'] # get total number of tmax days -- CPT 2020.02.23

        df = event_split(dates, ID_HDC_G0, intensity, tmax)# , total_days) #CPT 2020.02.23

        df_out = df_out.append(df)

    return df_out

In [14]:
def jul_convert(dates):
    "Function turn days into julian datetime"
    jul_days = pd.to_datetime(dates).to_julian_date()
    
    return jul_days

## Ok let's see how this works

In [7]:
# Make data into an xr.DataArray
df_id = list(df1983['ID_HDC_G0'])
date_list = list(df1983.columns[3:])
df_temp = df1983.iloc[:,3:] # get only temp columns
df_temp_drop = df_temp.dropna() # drop cities with no temp record
space_dim = 'ID_HDC_G0'
time_dim = 'date'

tmax_arr = df_temp_drop.to_numpy()

tmax_xr_da = xr.DataArray(tmax_arr, coords=[df_id, date_list], 
                     dims=[space_dim, time_dim])

In [11]:
df1983_406 = tmax_days(tmax_xr_da, 40.6)
df1983_406.head(1)

,ID_HDC_G0,dates,tmax,tmax_tntensity
0,3091,[1983.05.16],[42.49354861557129],[1.8935486155712908]


In [22]:
df1983_406_STATS = tmax_stats(df1983_406)

In [23]:
df1983_406_STATS.dtypes

ID_HDC_G0          int64
year              object
duration           int64
avg_temp         float64
avg_intensity    float64
tot_intensity    float64
event_dates       object
intensity         object
tmax              object
dtype: object

In [28]:
for item in df1983_406_STATS.iloc[0,:].values:
    print(type(item))

<class 'numpy.int64'>
<class 'str'>
<class 'numpy.int64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


## try json for 2 day

In [31]:
fn_nm = DATA_PATH+'interim/jsontest.json'
df1983_406_STATS.to_json(fn_nm, orient = 'split', index = False)

In [35]:
import json
with open(fn_nm) as json_data:
    data = json.load(json_data)

In [38]:
df = pd.read_json(fn_nm, orient='split')

In [46]:
type(df['event_dates'][0])

list

In [72]:
def find_two(df, fn_out):
    
    "find two day events buddy"
    print(len(df))
    df_out = df[df['duration'] > 1]
    print(len(df_out))
    #df_out.to_json(fn_out, orient = 'split', index = False)

In [55]:
fn_out = DATA_PATH+'interim/jsontest2day.json'
find_two(df, fn_out)

157540
102552


In [77]:
# as a loop
fns = sorted(glob(DATA_PATH+'interim/ERA5_STATS/*csv'))


for fn in fns:
    
    fn_out = fn.split('STATS/')[1].split('.csv')[0]
    fn_out = fn_out+'-2d406.json'
    print(fn_out)
    
    df = pd.read_csv(fn)
    print(len(df))
    df_out = df[df['duration'] > 1]
    print(len(df_out))
#     df_out = find_two(fn, fn_out)
#     print('\n')

GHS-ERA5-Stats_1983-2d406.json
157540
102552
GHS-ERA5-Stats_1984-2d406.json
167920
109274
GHS-ERA5-Stats_1985-2d406.json
163842
105745
GHS-ERA5-Stats_1986-2d406.json
165723
107579
GHS-ERA5-Stats_1987-2d406.json
164761
108021
GHS-ERA5-Stats_1988-2d406.json
171581
112544
GHS-ERA5-Stats_1989-2d406.json
161433
104770
GHS-ERA5-Stats_1990-2d406.json
173922
114781
GHS-ERA5-Stats_1991-2d406.json
162674
104662
GHS-ERA5-Stats_1992-2d406.json
157817
103797
GHS-ERA5-Stats_1993-2d406.json
165024
107132
GHS-ERA5-Stats_1994-2d406.json
174729
114889
GHS-ERA5-Stats_1995-2d406.json
169289
110326
GHS-ERA5-Stats_1996-2d406.json
167882
110057
GHS-ERA5-Stats_1997-2d406.json
171199
111914
GHS-ERA5-Stats_1998-2d406.json
171928
114703
GHS-ERA5-Stats_1999-2d406.json
177208
116065
GHS-ERA5-Stats_2000-2d406.json
164259
109016
GHS-ERA5-Stats_2001-2d406.json
164007
108755
GHS-ERA5-Stats_2002-2d406.json
168388
110843
GHS-ERA5-Stats_2003-2d406.json
173438
114242
GHS-ERA5-Stats_2004-2d406.json
166382
110353
GHS-ERA5-S

## Ok now find 115

In [78]:
df = pd.read_csv(fns[0])

In [105]:
def tmax_float(tmax):
    tmaxfloat = float(tmax.split('[')[1].split(']')[0])
    return tmaxfloat

In [118]:
def test(value):
    out = value
    out = float(out.split('[')[1].split(']')[0])
    return out

In [119]:
# facts['pop2050'] = facts.apply(lambda row: final_pop(row['population'],row['population_growth']),axis=1)
df.apply(lambda row: test(row['tmax']), axis = 1)

ValueError: ("could not convert string to float: '41.90553678 40.79086727'", 'occurred at index 24')

In [ ]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('factbook.db')
query = "select * from facts where area_land =0;"
facts = pd.read_sql_query(query,conn)
print(list(facts.columns.values))

def final_pop(initial_pop,growth_rate):
    final = initial_pop*math.e**(growth_rate*35)
    return(final)

facts['pop2050'] = facts['population','population_growth'].apply(final_pop,axis=1)
facts['pop2050'] = facts.apply(lambda row: final_pop(row['population'],row['population_growth']),axis=1)
